# VUE to **wildflow**

Loads acoustic tag data (downloaded from VUE software) to [**wildflow**](https://wildflow.ai).

<img src="https://wildflow.ai/_next/static/media/whale.22046cae.jpg" width="300">


---
# 1. Setup your project details

- Please make sure that the details below are correct;
- And then run the cell:


In [ ]:
project = 'wildflow-pelagic' #@param {type: "string"}
path = 'pelagioskakunja/provided-data' #@param {type: "string"}

print('Your files will be stored here:\n'+ f'https://console.cloud.google.com/storage/browser/{path}')

---
# 2. [Optional] Load files from your computer to **wildflow** bucket.

In [ ]:
!pip install -q google-cloud-storage

from google.colab import files, auth
from google.cloud import storage
from IPython.display import display, Markdown, HTML
import os

# Authenticate with GCP
auth.authenticate_user()

# Initialize the GCS client
client = storage.Client(project=project)
bucket_name, folder_path = path.split('/', 1)
bucket = client.get_bucket(bucket_name)

# Upload files
uploaded = files.upload()

uploaded_gcs_paths = []
for fn, content in uploaded.items():
    blob = bucket.blob(f'{folder_path}/{fn}')
    blob.upload_from_string(content)
    uploaded_gcs_paths.append(f"gs://{bucket_name}/{folder_path}/{fn}")
    # Remove the file from the local Colab environment
    os.remove(fn)

# Display uploaded file paths
display(Markdown("### Uploaded Files:"))
for file_path in uploaded_gcs_paths:
    display(Markdown(f"- {file_path}"))

# Generate and display the GCS link
gcs_link = f'https://console.cloud.google.com/storage/browser/{path}'
display(HTML(f'Your files have been stored <a href="{gcs_link}" target="_blank">here</a>.'))


# 3. Load VUE file from the bucket to **wildflow** BigQuery.

In [ ]:
filename = 'pelagioskakunja/provided-data/C.Leucas_Caribe_Sergei.csv' #@param {type: "string"}
tablename = 'wildflow-pelagic.env.new_table' #@param {type: "string"}

from google.cloud import bigquery
from google.colab import auth
auth.authenticate_user()

# Standard schema fro VUE files.
schema = [
    {"name": "row_id", "type": "STRING", "mode": "NULLABLE"},
    {"name": "x1", "type": "STRING", "mode": "NULLABLE"},
    {"name": "x", "type": "STRING", "mode": "NULLABLE"},
    {"name": "date_and_time_utc", "type": "STRING", "mode": "NULLABLE"},
    {"name": "receiver", "type": "STRING", "mode": "NULLABLE"},
    {"name": "transmitter", "type": "STRING", "mode": "NULLABLE"},
    {"name": "transmitter_name", "type": "STRING", "mode": "NULLABLE"},
    {"name": "transmitter_serial", "type": "STRING", "mode": "NULLABLE"},
    {"name": "sensor_value", "type": "STRING", "mode": "NULLABLE"},
    {"name": "sensor_unit", "type": "STRING", "mode": "NULLABLE"},
    {"name": "station_name", "type": "STRING", "mode": "NULLABLE"},
    {"name": "latitude", "type": "STRING", "mode": "NULLABLE"},
    {"name": "longitude", "type": "STRING", "mode": "NULLABLE"},
    {"name": "transmitter_type", "type": "STRING", "mode": "NULLABLE"},
    {"name": "sensor_precision", "type": "STRING", "mode": "NULLABLE"},
    {"name": "latitude_2", "type": "STRING", "mode": "NULLABLE"},
    {"name": "longitude_2", "type": "STRING", "mode": "NULLABLE"}
]

project_id, dataset_id, table_id = tablename.split('.')
client = bigquery.Client(project=project_id)
dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)

job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
job_config.schema = [
    bigquery.SchemaField(field["name"], field["type"], mode=field["mode"]) for field in schema
]

uri = f"gs://{filename}"
load_job = client.load_table_from_uri(uri, table_ref, job_config=job_config)
load_job.result()

print(f"Loaded {load_job.output_rows} rows into {dataset_id}:{table_id}.")



In [ ]:
sql = """
SELECT latitude, longitude, COUNT(*) as rows_count
FROM `{}`
GROUP BY 1, 2
""".format(tablename)

df = client.query(sql).to_dataframe()

df.head()



In [ ]:
import folium

df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=10)

for index, row in df.iterrows():
    folium.CircleMarker(
        location=(row['latitude'], row['longitude']),
        radius=5,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(m)

m